# IFT6285 (TALN) 

## Devoir 3: Correction des mots

Réalisé par:

Mouna Dhaouadi 

Kacem Khaled

Soumis à:

Prof. Philippe Langlais

Frédéric Piedboeuf


**Automne 2021**

In [ ]:
!pip install editdistance

In [ ]:
!pip install soundex

In [ ]:
!pip install textdistance

In [ ]:
import jaro
import heapq as hq
import editdistance
import nltk
import soundex
import textdistance
from soundex import Soundex
import pandas as pd

In [ ]:
lexique = 'voc-1bwc.txt'
wrong_words = 'test-mouna.txt'
reference = 'devoir3-train.txt'

## Question 1 : Correction

In [ ]:
def get_frequency_table(file_name):
    frequency_table = {}
    # create a dict from lexique
    with open(file_name ,'r',encoding="utf8") as f:
        lines = f.readlines()
        for line in lines:
            s = line.rstrip('\n').lstrip()
            frequency = int(s.split(' ')[0])
            word = s.split(' ')[1]
            frequency_table[word] = frequency
    f.close()
    return frequency_table

In [ ]:
def find_best_candidates(wrong_word, vocab_scores, nb_best_scores, largest, n=5, ordre= 'distance'): 
    scores = list(set(a[0] for a in vocab_scores.values())) 
    if largest:
     # Find three biggest scores (sorted) --> three best possible corrections
        best_scores= hq.nlargest(nb_best_scores, scores)
    else:
        best_scores=  hq.nsmallest(nb_best_scores, scores)
            
    best_candidates = {v:vocab_scores[v] for v in vocab_scores if vocab_scores[v][0] in best_scores}
    
    if ordre == 'distance':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][0]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]
    elif ordre == 'unigram':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][1]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]
    elif ordre == 'comb_d_u':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][0],best_candidates[k][1]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]
    elif ordre == 'comb_u_d':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][1],best_candidates[k][0]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]


    affichage(wrong_word, corrections)
    
    return corrections, best_candidates

In [ ]:
def affichage(wrong_word, corrections):
    if len(corrections) == 0 : 
        result =  wrong_word + '\t' + wrong_word
    else:
        result =  wrong_word + '\t' + '\t'.join(corrections)
    print(result)

In [ ]:
def process(dispatcher,distance_name,wrong_word,frequency_table,vocab):
    if distance_name=="Soundex":
        # We want Englidh corrections --> remove non english corrections (-1)
        vocab = [ v for v in vocab if instance.compare(v, wrong_word) >= 0 ]
    distance_scores = { v: [dispatcher[distance_name](v, wrong_word), frequency_table[v]] for v in vocab }
    print(f'Corrections using {distance_name} distance :')
    jaro_winkler_corrections,_ = find_best_candidates(wrong_word, distance_scores, 3, False,5, ordre= 'unigram') # 'distance' , 'unigram' or 'comb_d_u' or 'comb_u_d'


In [94]:
def corrige(wrong_words, lexique_file, distances=['Jaro_Winkler','Levenshtein']):

    frequency_table = get_frequency_table(lexique_file)
    vocab = list(frequency_table.keys())
    instance = Soundex()

    # https://pypi.org/project/textdistance/
    dispatcher = {
        'Jaro_Winkler' : textdistance.jaro_winkler.distance,
        'Levenshtein' : textdistance.levenshtein.distance,
        'Jaccard': textdistance.jaccard.distance,
        'Cosine': textdistance.cosine.distance,
        'Hamming': textdistance.hamming.distance,
        'LCSS': textdistance.lcsstr.distance,
        'Damerau_Levenshtein': textdistance.damerau_levenshtein.distance,
        'Needleman_Wunsch': textdistance.needleman_wunsch.distance,
        'Soundex': instance.compare
                 }
    

    # lit une liste de mots a corriger, un par ligne
    with open(wrong_words, 'r',encoding="utf8") as f:
        for line in f:

            wrong_word = line.rstrip().split()[0]
            print('********************************* ' + wrong_word)
            # calculate the distance for all the vocabulary
            # for each word in vocabulary, create a dictionary of candidates with 'nb_best_scores': {cand1:score1,cand2:score2,...}
            # get the 'n' best candidates and order them according to a criteria: 'distance' , 'unigram' or 'comb_d_u' or 'comb_u_d'
            
            for d in distances:
                process(dispatcher,d,wrong_word,frequency_table,vocab)

    f.close()
    

In [95]:
# distances = ['Jaro_Winkler', 'Levenshtein', 'Jaccard', 'Cosine', 'Hamming', 'LCSS', 'Damerau_Levenshtein', 'Needleman_Wunsch', 'Soundex']
distances = ['Jaro_Winkler', 'Levenshtein', 'Jaccard', 'Cosine', 'Hamming', 'LCSS', 'Damerau_Levenshtein', 'Needleman_Wunsch', 'Soundex']
corrige(wrong_words, lexique,distances)


********************************* aple
Corrections using Jaro_Winkler distance :
aple	apley	aple	apple
Corrections using Levenshtein distance :
aple	acle	applet	aslo	hplc	kable
Corrections using Jaccard distance :
aple	apley	palea	alpen	apeal	apfel
Corrections using Cosine distance :
aple	apley	palea	alpen	apeal	apfel
Corrections using Hamming distance :
aple	acle	aslo	hplc	odle	vpls
Corrections using LCSS distance :
aple	acle	erap	hplc	kapi	leet
Corrections using Damerau_Levenshtein distance :
aple	acle	applet	aslo	hplc	kable
Corrections using Needleman_Wunsch distance :
aple	acle	applet	aslo	hplc	kable
Corrections using Soundex distance :
aple	aaaa	aaa-rating	aaoifi	aarti	abaca
********************************* juce
Corrections using Jaro_Winkler distance :
juce	juche	juckes	juice
Corrections using Levenshtein distance :
juce	acle	auh	bbcs	boice	buker
Corrections using Jaccard distance :
juce	uce	ujc	jue	ceu	judice
Corrections using Cosine distance :
juce	uce	ujc	jue	ceu	judice
Corre

In [96]:
textdistance.cosine.distance('test', 'text') 

0.25

In [ ]:
# Votre programme prendra comme seul argument obligatoire le nom d'un lexique de mots 
# Vous pouvez doter votre programme d'options permettant de contr^oler le comportement de votre correcteur. 

# TODO a seperate python script that we can run with the provided command in TP
import argparse

order_by = ['distance','unigram','comb_d_u','comb_u_d']

parser = argparse.ArgumentParser(description='Words correction')
parser.add_argument('-d', '--distance', default='Jaro_Winkler',
                    choices=distances,
                    help='Distances: ' +
                        ' | '.join(distances) +
                        ' (default: Jaro_Winkler)')

parser.add_argument('-o', '--order', default='unigram',
                    choices=order_by,
                    help='Distances: ' +
                        ' | '.join(order_by) +
                        ' (default: unigram)')

def main():
    args = parser.parse_args()
    #args.distance
    #args.order
    corrige(reference, lexique,args.distance)

if __name__ == '__main__':
    main()

In [ ]:
corrige(reference, lexique,['Jaro_Winkler'])

## Evaluation

In [ ]:
reference = 'devoir3-train.txt'

In [93]:
def eval(correction_line, reference):
    # look for wrong word in the reference file
    
    # take the correct correctionss
    
    # compute mesure de qualite ? 
    pass
    